# Read Full df

In [1]:
import numpy as np

from energy_model.models.system_energy_model import SystemEnergyModel
from energy_model.energy_model_parameters import FULL_DATASET_BEFORE_PROCESSING_PATH
import pandas as pd
from energy_model.models.process_energy_model import ProcessEnergyModel


full_df = pd.read_csv(FULL_DATASET_BEFORE_PROCESSING_PATH, index_col=0)

# Details about the data

In [2]:
full_df.shape

(17415, 56)

# Helpful utils

In [3]:
def create_system_model():
    system_model = SystemEnergyModel()
    system_model.build_energy_model(full_df)
    return system_model

In [4]:
def create_process_model(sys_model):
    process_energy_model = ProcessEnergyModel(sys_model)
    process_energy_model.build_energy_model(full_df)
    return process_energy_model

In [5]:
def run_system_test_differences(x_test, y_test, model):
    y_pred = model.predict(x_test)
    diff = y_pred - y_test
    all_results = {}
    for estimated_energy_process in [0.5, 1, 2.5, 5, 10]:
        all_results[estimated_energy_process] = len([d for d in diff if d > estimated_energy_process])

    print(all_results)


In [6]:
def read_test_set() -> tuple:
    test_set = pd.read_csv("test_energy_consumption_system_mWh.csv")
    y_real = test_set['target']
    x_test = test_set.drop('target', axis=1)
    return x_test, y_real

# Current Model (no-changes)

In [7]:
system_original_model = create_system_model()


--- Running Fold 1/5 ---
Model's configuration: 
{'loss': 'squared_error', 'max_depth': 16, 'min_samples_leaf': 10, 'max_iter': 400, 'l2_regularization': 0.1}


*** Model's Accuracy on test-set ***
Symmetric Mean Absolute Percentage Error (Symmetric MAPE) value: 13.677025670484575
Relative Root Mean Squared Error (Relative RMSE) value: 6.1629013877880405
Standard Deviation (Std.) value: 25.291373074543824
Average Percentage Error Rate (Average PER) value: 9.566913287619673
Mean Squared Error (MSE) value: 3.956272780007478
Root Mean Squared Error (RMSE) value: 1.9890381544876101
Mean Absolute Error (MAE) value: 1.204544309976508
Root Mean Squared Percent Error (RMSPE) value: 29.298275056955227
Percentile Squared Error - percentile = 90 value: 7.235551527490274
Percentile Squared Error - percentile = 95 value: 16.228029763712527
Tail Root Mean Squared Error Percentile 95 (Tail-RMSE) value: 6.991319212300731
Root Mean Squared Error Ratio for Percentile 95 value: 3.514924636577292
New bes

In [ ]:
rmse_ratio = 11.529 / 3.29
rmse_ratio

In [8]:
x_test_original, y_real_original = read_test_set()

In [9]:
import numpy as np
normalized_tail_rmse = 11.529 / np.std(y_real_original)
normalized_tail_rmse

np.float64(0.37878431400720786)

In [10]:
from energy_model.models.persistence_manager import PersistenceManager

PersistenceManager.save_model(system_original_model, f"ratio_rmse_based_system_energy_prediction_model.pkl")

In [12]:
process_original_model = create_process_model(system_original_model)


--- Running Fold 1/5 ---
Model's configuration: 
{'loss': 'squared_error', 'learning_rate': 0.06, 'max_iter': 300, 'max_depth': 11, 'min_samples_leaf': 20, 'l2_regularization': 0.05, 'max_bins': 255, 'early_stopping': True, 'validation_fraction': 0.1, 'n_iter_no_change': 20, 'random_state': 42}


Negative predictions found:
  Values : [-0.8904183815789821, -1.0117342024976972, -1.5231531084425631, -0.23615944469363115, -0.8011356783410305, -1.1394670578954023, -0.5717655317733884, -0.9381782696390348, -1.0570338220127367, -0.16570185121755318, -0.7945338334847019, -1.7079621827672444, -0.5353112362269685, -0.014271753526757453, -0.2086260479349609, -1.940680600812702, -2.1308974807901264, -0.1406229176431666, -0.5943610714077371, -1.6280044352149077, -0.46263642057176685, -0.043167204900697205, -0.3776969530467742, -0.22318091493493208, -0.1423417507121173, -2.6324507567215996, -0.44845818446622576, -0.5021469629985168, -0.2848187841798382, -0.8228841846297332, -0.04630661788960177, -

In [11]:
run_system_test_differences(x_test_original, y_real_original, system_original_model)

{0.5: 1011, 1: 671, 2.5: 179, 5: 56, 10: 8}


# Huber Mode Model

In [ ]:
system_huber_model = create_system_model()

In [ ]:
process_huber_model = create_process_model(system_huber_model)

In [ ]:
from energy_model.models.persistence_manager import PersistenceManager

PersistenceManager.save_model(process_huber_model, f"smape_based_process_energy_prediction_model_diff_params.pkl")

In [ ]:
x_test_huber, y_real_huber = read_test_set()

In [ ]:
run_system_test_differences(x_test_huber, y_real_huber, system_huber_model)